In [1]:
from utils import get_images_from_directory, get_images_from_directory_keyword
from matplotlib import pyplot as plt
from models import inceptionV3
import numpy as np
import sys, os
from helpers import *

2023-07-04 15:59:33.491549: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-04 15:59:41.661107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sathlerar/data/conda/lib:/usr/local/CUDA/11.3.0/bin:/usr/local/CUDA/11.3.0/targets/x86_64-linux/lib:/usr/local/cuDNN/8.2.1/CUDA-11.3/lib64:/usr/bin
2023-07-04 15:59:41.661248: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or dire

In [2]:
from interface import running_parameters_interface
nb_runnings = 1
J_ROOT = os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID'])
parameters = running_parameters_interface(nb_runnings, J_ROOT)

Input directory


FileChooser(path='/gpfs/gsfs8/users/sathlerar/pecot', filename='', title='', show_hidden=False, select_desc='S…

Input model


FileChooser(path='/gpfs/gsfs8/users/sathlerar/pecot', filename='', title='', show_hidden=False, select_desc='S…

Output directory


FileChooser(path='/gpfs/gsfs8/users/sathlerar/pecot', filename='', title='', show_hidden=False, select_desc='S…

In [3]:
def running(nb_runnings, parameters):
    for i in range(nb_runnings):
        if parameters[0][i].selected==None:
            sys.exit("Running #"+str(i+1)+": You need to select an input directory for images to be processed")
        if parameters[1][i].selected==None:
            sys.exit("Running #"+str(i+1)+": You need to select a trained model to process your images")
        if parameters[2][i].selected==None:
            sys.exit("Running #"+str(i+1)+": You need to select an output directory for processed images")

        print(f"n_features: {parameters[5][i].children[1].value}\n" \
              + f"n_channels: {parameters[4][i].children[1].value}\n" \
              + f"dimx: {parameters[6][i].children[1].value}\n" \
              + f"dimy: {parameters[7][i].children[1].value}\n" \
              + f"weights_path: {parameters[1][i].selected}")
        
        model = inceptionV3(n_features=parameters[5][i].children[1].value, 
                            n_channels=parameters[4][i].children[1].value,
                            dimx=parameters[6][i].children[1].value, 
                            dimy=parameters[7][i].children[1].value, 
                            weights_path=parameters[1][i].selected)
        
        print(f"data_location: {parameters[0][i].selected}\n" \
              + f"output_location: {parameters[2][i].selected}\n" \
              + f"bs: {parameters[8][i].children[1].value}\n" \
              + f"mask_names: {parameters[3][i].children[1].value}\n" \
              + f"normalization: {parameters[9][i].children[1].value}")
        
        run_models_on_directory(parameters[0][i].selected, parameters[2][i].selected, 
                                model, bs=parameters[8][i].children[1].value, maxDim=800, 
                                mask_names=parameters[3][i].children[1].value, 
                                normalization=parameters[9][i].children[1].value)
        del model
        return stacked_images, image_list

In [4]:
def run_models_on_directory(data_location, output_location, model, bs=32, maxDim=800, 
                            mask_names='None', normalization="nuclei segmentation"):

    # determine the number of channels and classes as well as the imaging field dimensions
    input_shape = model.layers[0].output_shape
    n_channels = input_shape[0][1]
    imaging_field_x = int((input_shape[0][1]-1)/2)
    imaging_field_y = int((input_shape[0][2]-1)/2)
    output_shape = model.layers[-1].output_shape
    n_classes = output_shape[-1]

    # determine the image size
    image_size_x, image_size_y, nb_chan = get_image_sizes(data_location)
    print(f"Image x: {image_size_x}\n" \
          + f"Image y: {image_size_y}\n" \
          + f"Image Channels: {nb_chan}\n")
    
    # process images
    cpt = 0
    model_output = []
    processed_image_list = run_model_on_directory_pixByPix(data_location, mask_names, output_location, model, 
                                                         win_x = imaging_field_x, win_y = imaging_field_y, 
                                                         bs=bs, maxDim=maxDim, normalization = normalization)

    model_output += [np.stack(processed_image_list, axis = 0)]

    return model_output, processed_image_list

In [17]:
def run_model_on_directory_pixByPix(data_location, mask_names, output_location, model, 
                                    win_x = 30, win_y = 30, bs=32, maxDim=800, 
                                    normalization="nuclei segmentation"):
    
    n_classes = model.layers[-1].output_shape[-1]
    counter = 0

    if mask_names == 'None':
        image_list, image_names = get_images_from_directory_w_padding(data_location)
    else:
        image_list, image_names = get_images_from_directory_keyword(data_location, mask_names, True)
    processed_image_list = []

    if mask_names != "None":
        mask_list, image_names = get_images_from_directory_keyword(data_location, mask_names)
    
    for idx, img in enumerate(image_list):
        print(f"Processing image {idx + 1} of {len(image_list)}: {image_names[idx]}")
        if mask_names == "None":
            processed_image = run_model_pixByPix(img, model, win_x = win_x, win_y = win_y, 
                                                 bs=bs, maxDim=maxDim, normalization = normalization)
        else:
            processed_image = run_model_pixByPixOnMasks(img, mask_list[counter], model, win_x = win_x, win_y = win_y, 
                                                        bs=bs, maxDim=maxDim, normalization = normalization)
        processed_image_list += [processed_image]

        # Save images
        for i in range(n_classes):
            cnnout_dir = os.path.join(output_location, image_names[idx])
            if not os.path.isdir(cnnout_dir):
                os.mkdir(cnnout_dir)
            cnnout_name = os.path.join(cnnout_dir, f"image_c{i}.tif")
            tiff.imwrite(cnnout_name, processed_image[:,:,i])
    
    return processed_image_list

In [18]:
def run_model_pixByPix(img, model, win_x = 30, win_y = 30, std = False, split = True, process = True, bs=32, 
                       maxDim=800, normalization = "nuclei segmentation"):                           
    if normalization == "nuclei segmentation":
        for j in range(img.shape[-1]):
            img[0,:,:,j] = process_image(img[0,:,:,j], win_x, win_y)
    else:
        for j in range(img.shape[-1]):
            img[0,:,:,j] = process_image_onlyLocalAverageSubtraction(img[0,:,:,j], win_x, win_y)
        
    img = np.pad(img, pad_width = [(0,0), (win_x, win_x), (win_y,win_y), (0,0)], mode = 'reflect')
    n_classes = model.layers[-1].output_shape[-1]
    image_size_x = img.shape[1]
    image_size_y = img.shape[2]
    model_output = np.zeros((image_size_x-2*win_x,image_size_y-2*win_y,n_classes), dtype = np.float32)

    print(f"Input image size: {img.shape[1]}x{img.shape[2]}\n" \
          + f"Image size from model: {image_size_x}x{image_size_y}\n" \
          + f"Image size of stitch: {model_output.shape[0]}x{model_output.shape[1]}")
        
    x_minIterator, y_minIterator = win_x, win_y
    x_maxIterator = min(image_size_x, maxDim) - win_x
    y_maxIterator = min(image_size_y, maxDim) - win_y
    
    while x_minIterator<(image_size_x-win_x) and y_minIterator<(image_size_y-win_y):
        test_images = []
        for x in range(x_minIterator, x_maxIterator):
            for y in range(y_minIterator, y_maxIterator):
                test_images.append(img[0,x-win_x:x+win_x+1,y-win_y:y+win_y+1,:])
               
        test_images = np.asarray(test_images)
        test_images = test_images.astype('float32')

        predictions = model.predict(test_images, verbose=1, batch_size=bs)

        cpt = 0
        print(f"Iterating...")
        for x in range(x_minIterator, x_maxIterator):
            for y in range(y_minIterator, y_maxIterator):
                model_output[x-win_x,y-win_y,:] = predictions[cpt,:]
                cpt += 1

        print(f"Adjusting Iterator...")
        if x_maxIterator < image_size_x-win_x:
            x_minIterator = min(x_maxIterator,image_size_x)
            if image_size_x-x_minIterator < maxDim:
                x_maxIterator = image_size_x-win_x
            else:
                x_maxIterator = x_minIterator+maxDim-win_x
        else:       
            x_minIterator = win_x
            x_maxIterator = min(image_size_x,maxDim)-win_x
            y_minIterator = min(y_maxIterator,image_size_y)
            if image_size_y-y_minIterator < maxDim:
                y_maxIterator = image_size_y-win_y
            else:
                y_maxIterator = y_minIterator+maxDim-win_y

    return model_output

In [19]:
stack, img_list = running(nb_runnings, parameters)

n_features: 3
n_channels: 1
dimx: 65
dimy: 65
weights_path: /gpfs/gsfs8/users/sathlerar/pecot/InceptionV3/models/nucleiSegmentation/2023-07-04_InceptionV3_1_ch_3_cl_65_65_lr_0.01_withDA_10_ep.h5
data_location: /gpfs/gsfs8/users/sathlerar/pecot/for_anal/drg_data/
output_location: /gpfs/gsfs8/users/sathlerar/pecot/for_anal/23-07-04_32x32_out/
bs: 32
mask_names: None
normalization: nuclei segmentation
Image x: 1132
Image y: 1132
Image Channels: 1

Processing image 1 of 39: 230217mtDNAwithTom20-(5)_Young_i3


IndexError: too many indices for array: array is 2-dimensional, but 4 were indexed